In [1]:
import hazm
import numpy as np
import pandas as pd
from hazm import sent_tokenize, word_tokenize
import nltk
from nltk.tokenize import word_tokenize as eng_tokenize
import pickle
import json
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from tqdm.auto import tqdm

from torch.utils.data import Dataset
import torch.nn.functional as F
from tensorflow.keras.utils import pad_sequences
from sklearn.preprocessing import MinMaxScaler
import keras

2024-05-13 17:28:46.612629: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-13 17:28:46.694325: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-13 17:28:46.694430: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-13 17:28:46.694466: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-13 17:28:46.707028: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: A

In [72]:
class dataset(Dataset):
    def __init__(self,en_data,fa_data,en_index,fa_index):
        super(dataset,self).__init__()
        for seq in en_data:
            for i in range(len(seq)):
                seq[i]=en_index[seq[i]]
        for seq in fa_data:
            for i in range(len(seq)):
                seq[i]=fa_index[seq[i]]
        self.en_data=torch.from_numpy(pad_sequences(en_data,padding="post",maxlen=10,value=1))
        self.fa_data=torch.from_numpy(pad_sequences(fa_data,padding="post",maxlen=10,value=1))
        self.fa_data=torch.concat((torch.zeros(len(self.fa_data),1),self.fa_data),dim=1).long()
    def __len__(self):
        return len(self.en_data)
    def __getitem__(self,idx):
        return self.en_data[idx],self.fa_data[idx][:-1],self.fa_data[idx][1:]

In [3]:
with open("./hugg_preprocessed/vocab_en", "rb") as fp:   # Unpickling
    vocab_en = pickle.load(fp)
with open("./hugg_preprocessed/vocab_fa", "rb") as fp:   # Unpickling
    vocab_fa = pickle.load(fp)
with open("./hugg_preprocessed/dataset_en", "rb") as fp:   # Unpickling
    dataset_en = pickle.load(fp)
with open("./hugg_preprocessed/dataset_fa", "rb") as fp:   # Unpickling
    dataset_fa = pickle.load(fp)
with open("./hugg_preprocessed/en_index.json", "r") as fp:   #Pickling
    en_index=json.load(fp)
with open("./hugg_preprocessed/fa_index.json", "r") as fp:   #Pickling
    fa_index=json.load(fp)

In [23]:
class CELL(nn.Module):
    def __init__(self,hidden_size,embeding_size):
        super(CELL,self).__init__()
        # self.input_net=nn.Embedding(num_embeddings=len(vocab),embedding_dim=embeding_dim)
        
        self.WF=nn.Parameter(torch.rand(hidden_size+embeding_size,hidden_size),requires_grad=True)
        self.BF=nn.Parameter(torch.rand(1,hidden_size),requires_grad=True)
        self.sigF=nn.Sigmoid()
        
        self.WI1=nn.Parameter(torch.rand(hidden_size+embeding_size,hidden_size),requires_grad=True)
        self.BI1=nn.Parameter(torch.rand(1,hidden_size),requires_grad=True)
        self.sigI=nn.Sigmoid()
        self.WI2=nn.Parameter(torch.rand(hidden_size+embeding_size,hidden_size),requires_grad=True)
        self.BI2=nn.Parameter(torch.rand(1,hidden_size),requires_grad=True)
        self.tanhI=nn.Tanh()
        
        self.WO=nn.Parameter(torch.rand(hidden_size+embeding_size,hidden_size),requires_grad=True)
        self.BO=nn.Parameter(torch.rand(1,hidden_size),requires_grad=True)
        self.tanhO=nn.Tanh()
        self.sigO=nn.Sigmoid()
    
    def forward(self,x_batch,short_memory,long_memory):
        """
        x_batch = (batch_size,embeding_size)
        short_memory =(batch_size,hidden_size)
        long_memory =(batch_size,hidden_size)
        """
        # emb_batch=self.input_net(x_batch)
        emb_batch=x_batch
        scaler=emb_batch.shape[0]
        #Forget gate
        new_batch=torch.concat((short_memory,emb_batch),dim=1) #(batch_size,hidden_size+embeding_size)
        zF=torch.matmul(new_batch,self.WF) +self.BF #batch_size,hidden_size
        aF=self.sigF(zF)
        
        #Input gate
        zI1=torch.matmul(new_batch,self.WI1) + self.BI1 #batch_size,hidden_size
        aI1=self.sigI(zI1)
        
        zI2=torch.matmul(new_batch,self.WI2) +self.BI2 #batch_size,hidden_size
        aI2=self.tanhI(zI2)
        aI=aI1*aI2
        
        #Output gate
        long_memory=(long_memory*aF)+(long_memory+aI) #batch_size,hidden_size
        
        zO1=torch.matmul(new_batch,self.WO) +self.BO #batch_size,hidden_size
        aO1=self.sigO(zO1)
        
        aO2=self.tanhO(long_memory)
        
        short_memory=aO1*aO2
        
        return short_memory,long_memory


In [51]:
class LSTM(nn.Module):
    def __init__(self,layer_num,hidden_size,embeding_size):
        super(LSTM,self).__init__()
        layers=[]
        for i in range(layer_num):
            layers.append(CELL(hidden_size,embeding_size))
            layers.append(nn.Linear(in_features=hidden_size,out_features=embeding_size))
        self.Pipeline=nn.ParameterList(layers[:-1])
    def forward(self,x,memory_cache):
        new_memory_cache=[]
        for i,l in enumerate(self.Pipeline):
            if((i+1)%2!=0):
                h,c=memory_cache[i//2][0],memory_cache[i//2][1]
                h_new,c_new=self.Pipeline[i](x,h,c)
                new_memory_cache.append([h_new,c_new])
                x=h
            else:
                x=self.Pipeline[i](x)
        return new_memory_cache,x
class Encoder(nn.Module):
    def __init__(self,layer_num,hidden_size,embeding_dim,vocab_size):
        super(Encoder,self).__init__()
        self.embeding_layer=nn.Embedding(num_embeddings=vocab_size,embedding_dim=embeding_dim)
        self.lstm=LSTM(layer_num,hidden_size,embeding_dim)
    def forward(self,memory_cache,x_batch):
        """
        x_batch shape =(batch_size,) input batch of english words
        memory_cache shape = (layer_num,2) its a 2D list contains [h,c] 
        """
        embeded_batch=self.embeding_layer(x_batch)
        new_memory_cache , _=self.lstm(embeded_batch,memory_cache)
        return new_memory_cache
class Decoder(nn.Module):
    def __init__(self,layer_num,hidden_size,embeding_dim,vocab_size):
        super(Decoder,self).__init__()
        self.embeding_layer=nn.Embedding(num_embeddings=vocab_size,embedding_dim=embeding_dim)
        self.lstm=LSTM(layer_num,hidden_size,embeding_dim)
        self.linear=nn.Linear(hidden_size,vocab_size)
        self.softmax=nn.Softmax(dim=1)
    def forward(self,memory_cache,x_batch):
        """
        x_batch shape =(batch_size,) input batch of english words
        memory_cache shape = (layer_num,2) its a 2D list contains [h,c] 
        """
        embeded_batch=self.embeding_layer(x_batch)
        new_memory_cache , x=self.lstm(embeded_batch,memory_cache)
        x=self.linear(x)
        x=self.softmax(x)
        return new_memory_cache , x

In [80]:
def reset_memory(batch_size,hidden_size,layer_num):
    mem_list=[]
    for i in range(layer_num):
        mem_list.append([torch.zeros(batch_size,hidden_size),torch.zeros(batch_size,hidden_size)])
    return mem_list
def detach_memory(memory_cache):
    for i in len(memory_cache):
        memory_cache[i][0]=memory_cache[i][0].detach()
        memory_cache[i][1]=memory_cache[i][1].detach()
    return memory_cache
def Train(TrainLoader,batch_size,epochs,encoder,decoder,encoder_config,decoder_config,EncoderOptimizer,DecoderOptimizer,loss_fn):
    for ep in tqdm(range(epochs)):
        for en,fa,fa_target in tqdm(TrainLoader):
            memory_cache=reset_memory(batch_size=en.shape[0],
                                  hidden_size=encoder_config["hidden_size"],
                                  layer_num=encoder_config["layers_num"])

            # Encoder
            
            seq_length=en.shape[1]
            for i in range(seq_length):
                x_batch=en[:,i]
                new_memory_cache=encoder(memory_cache,x_batch)
                memory_cache=new_memory_cache

            # Decoder
            
            seq_length=fa.shape[1]
            loss=0
            for i in range(seq_length):
                x_batch=fa[:,i]
                y_batch=fa_target[:,i]
                new_memory_cache,y_pred=decoder(memory_cache,x_batch)
                loss+=loss_fn(input=y_pred,target=y_batch.reshape(-1))
            loss=loss/batch_size
            loss.backward()
            EncoderOptimizer.step()
            DecoderOptimizer.step()
            EncoderOptimizer.zero_grad()
            DecoderOptimizer.zero_grad()

In [4]:
hidden_size=256
embeding_dim=1
batch_size=32
layer_number=4
en_vocab_size=len(vocab_en)
fa_vocab_size=len(vocab_fa)
epochs=100
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [74]:
ds=dataset(dataset_en,dataset_fa,en_index,fa_index)

In [75]:
trainLoader=DataLoader(ds,batch_size=batch_size)

In [ ]:
encoder_config={"layers_num":2,"hidden_size":256,"embeding_size":100}
decoder_config={"layers_num":2,"hidden_size":256,"embeding_size":100}

encoder=Encoder(encoder_config["layers_num"],encoder_config["hidden_size"],encoder_config["embeding_size"],vocab_size=en_vocab_size)
decoder=Decoder(decoder_config["layers_num"],decoder_config["hidden_size"],decoder_config["embeding_size"],vocab_size=fa_vocab_size)
EncoderOptimizer=torch.optim.Adam(lr=0.0001,params=encoder.parameters()).to(de)
DecoderOptimizer=torch.optim.Adam(lr=0.0001,params=decoder.parameters())
loss_fn=nn.CrossEntropyLoss()
Train(TrainLoader=trainLoader,
      batch_size=batch_size,
      epochs=epochs,
      encoder=encoder,
      decoder=decoder,
      encoder_config=encoder_config,
      decoder_config=decoder_config,
      EncoderOptimizer=EncoderOptimizer,
      DecoderOptimizer=DecoderOptimizer,
      loss_fn=loss_fn
     )

In [36]:
lstm1=LSTM(1,2,3)
lstm2=LSTM(1,2,3)
oprtimizer=torch.optim.Adam(params=[
    {'params': lstm1.parameters()},
    {'params': lstm2.parameters()}
],lr=0.0001)
# lstm1.parameters()